In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [3]:
import pandas as pd


In [10]:
import sys
sys.path.append('../NER')
sys.path.append('..')

In [11]:
import bclm

In [5]:
import ne_evaluate_mentions as nem

In [6]:
import configs
import os

In [17]:
words = []
for n, ds in configs.datasets.items():
    for k in ds:
        if 'dev' in k or 'test' in k or 'train' in k:
            path = os.path.join(configs.data_folder, ds[k])
            print(path)
            for line in open(path, 'r'):
                word = line.split(' ')[0].strip()
                if word!='':
                    words.append(word)
                
len(words)

../NER/data/for_ncrf/morph_gold_train.bmes
../NER/data/for_ncrf/morph_gold_dev.bmes
../NER/data/for_ncrf/morph_gold_test.bmes
../NER/data/for_ncrf/token_gold_train_fix.bmes
../NER/data/for_ncrf/token_gold_dev_fix.bmes
../NER/data/for_ncrf/token_gold_test_fix.bmes
../NER/data/for_ncrf/token_gold_train_concat.bmes
../NER/data/for_ncrf/token_gold_dev_concat.bmes
../NER/data/for_ncrf/token_gold_test_concat.bmes
../NER/data/for_ncrf/morph_yap_dev.bmes
../NER/data/for_ncrf/morph_yap_test.bmes
../NER/data/for_ncrf/morph_yap_dev_pruned_all.bmes
../NER/data/for_ncrf/morph_yap_dev_pruned_non_o.bmes


435795

In [26]:
wc = Counter(words)
len(wc)

32852

In [27]:
wc.most_common(10)

[(',', 24414),
 ('ה', 23119),
 ('.', 20284),
 ('ב', 10314),
 ('של', 7281),
 ('"', 6025),
 ('ל', 5972),
 ('את', 5918),
 ('ו', 5394),
 ('על', 4557)]

In [42]:
ws = []
with open('data/htb_all_words.txt', 'w') as of:
    for w in wc.most_common():
        ws.append(w[0])
        of.write(w[0]+'\n')

In [43]:
ws[:10]

[',', 'ה', '.', 'ב', 'של', '"', 'ל', 'את', 'ו', 'על']

In [65]:
from gensim.models.fasttext import load_facebook_model


def get_embeddings(path, words):
    model = load_facebook_model(path)

    embs = {}

    for w in words:
        embedding_vector = model.wv[w]
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            # words not found in embedding index will be all-zeros.
            embs[w] = embedding_vector
    return embs

models = {
    'yap_ft_sg': 'wikipedia.alt_tok.yap_form.fasttext_skipgram.model.bin',
    'tok_ft_sg': 'wikipedia.alt_tok.tokenized.fasttext_skipgram.model.bin'
}

def get_model_path(n):
    return os.path.join('../wordembedding-hebrew/models', models[n])

In [60]:
yap_ft_sg = get_embeddings(get_model_path('yap_ft_sg'), ws)

In [61]:
' '.join([str(x) for x in yap_ft_sg['את']])

'-0.029589325 0.067772046 -0.09019217 0.07522869 -0.0018800125 -0.05052501 0.058455113 -0.10221776 0.060982287 -0.13587582 -0.13670188 0.0018101754 0.1430349 -0.11154884 0.1380201 -0.10028729 0.027101487 -0.11713677 -0.033129737 0.12475677 0.033628162 0.16784416 -0.16621032 0.0339112 -0.050597742 0.04204543 0.3073591 0.073530555 0.09733049 0.13907295 -0.09121367 0.017948449 -0.17232066 -0.08307035 -0.3000877 0.0434435 -0.024405256 -0.059446596 0.09396139 -0.22093344 -0.029940933 0.15892819 0.10566935 0.16188699 -0.07442418 -0.1677086 0.20364654 0.024425328 -0.2208995 0.17712471 -0.34308833 -0.20723511 -0.15393549 0.087095596 -0.018120155 0.31398413 0.03386666 0.023856193 -0.160196 0.029039055 -0.006198272 0.038065292 0.13167456 0.14024588 -0.018231846 -0.06053351 -0.13852328 0.10735424 -0.080169186 -0.104560524 -0.04723718 0.0036140084 0.03957808 -0.15783224 -0.042963784 -0.01711744 -0.0058503747 0.06964392 0.13633208 -0.06425118 -0.19311634 0.3463379 -0.0061738044 0.012622938 -0.00579

In [63]:
with open('data/htb_all_words.wikipedia.alt_tok.yap_form.fasttext_skipgram.txt', 'w') as of:
    for w, e in yap_ft_sg.items():
        of.write(w+' '+' '.join([str(x) for x in e])+'\n')

In [64]:
!head data/htb_all_words.wikipedia.alt_tok.yap_form.fasttext_skipgram.txt

, -0.12320192 0.011220977 -0.006417066 0.053529475 0.11791958 -0.017090425 0.00092396326 0.11228291 0.30503762 0.035435148 -0.118931256 0.082847685 0.06884778 0.051606093 -0.06750929 -0.18016735 -0.07057498 -0.23951693 0.021299258 0.1482788 0.06471552 0.1314311 0.09137564 0.054547228 0.018452203 0.019663334 0.1436525 0.036204927 0.07522814 -0.056511175 0.05077192 -0.07944211 0.06783289 0.06389449 0.07631259 0.087860264 0.041449368 0.027272483 -0.12122535 -0.20425971 0.009519778 0.16893607 -0.018135376 -0.025708772 0.09841262 -0.13414422 0.018760549 -0.029834505 -0.039748084 -0.018689677 0.02524279 -0.09561184 0.008445159 -0.03840214 0.13465199 -0.028207012 -0.09464779 -0.013414323 0.007566966 -0.008400662 -0.13995908 0.119904935 -0.067060865 0.13650498 0.050017264 -0.07531574 0.009797886 -0.1465854 0.02499311 -0.11508637 -0.13434602 -0.04606028 0.0018844828 -0.054428577 -0.06963893 0.043384336 -0.0022995546 0.10829651 0.032199856 0.0324893 -0.0002705846 0.022868805 0.04699978 0.0656832

In [66]:
tok_ft_sg = get_embeddings(get_model_path('tok_ft_sg'), ws)

In [68]:
with open('data/htb_all_words.wikipedia.alt_tok.tokenized.fasttext_skipgram.txt', 'w') as of:
    for w, e in tok_ft_sg.items():
        of.write(w+' '+' '.join([str(x) for x in e])+'\n')

In [69]:
!head data/htb_all_words.wikipedia.alt_tok.tokenized.fasttext_skipgram.txt

, 0.12976168 0.20693739 -0.16358475 0.0031254888 -0.01258507 0.1358625 0.11508744 0.058779083 0.089778796 0.12188145 -0.06650192 0.093571275 0.029144596 0.012928501 0.078496695 0.07437801 -0.0118843615 -0.084091395 0.008798871 -0.020393021 0.139122 -0.045989744 0.076088205 -0.09363521 -0.035057552 0.18813445 0.03715067 -0.013883289 -0.08187736 0.06139416 -0.09512491 -0.044352874 0.15219551 -0.084490225 -0.012149006 0.054951303 -0.086139254 -0.07203263 0.089777976 0.0013710633 0.027686555 -0.18475659 0.09745669 0.0058658253 0.01765578 -0.0038873162 0.018463925 -0.21964829 0.0025322847 0.05290205 -0.05973135 0.08615337 0.08329097 0.10944703 -0.19991732 -0.06564408 0.014486399 0.0038076611 -0.096289404 -0.11489778 0.07071455 -0.117611304 -0.13333897 0.10735941 0.09520378 0.012124084 0.12662466 0.11256027 0.12292387 0.08788182 -0.056947246 -0.015558548 -0.2054494 0.101145715 -0.064027116 -0.11095709 0.0066385977 -0.07092668 0.08194401 0.10948129 0.050789207 0.032290034 -0.013693001 -0.1103

In [83]:
import re
from collections import Counter
devwo = []
for line in open(os.path.join(configs.data_folder, configs.datasets['gold_morpheme']['dev_dir']), 'r'):
    word = line.split(' ')[0].strip()
    if word!='':
        devwo.append(word)
        
RE_D = re.compile('\d')
def hasnum(string):
    res = RE_D.search(string) is not None
    if res:
        print (string)
    return 

nums = Counter(map(hasnum, devwo))
nums

500
50
105
24
0192
111
16
1980
80
1988
200
25
48
10
500
20
95
63
1
2
51
3
1960
4
05
06
07
1977
5
6
02
70
80
60
82000
52000
7
50
1989
8
07
9
05
1979
1989
1
35
2
20
9
1990
32
21
1992
521
1992
1984
06
90
1984
58
1990
1988
9
84
7
1978
1942
1958
1983
1984
1972
1971
1983
1984
18
1200
1400
1500
2115
4
1
3


Counter({None: 11301})

In [77]:
83/11218

0.007398823319664825

In [85]:
import re
from collections import Counter
trwo = []
for line in open(os.path.join(configs.data_folder, configs.datasets['gold_morpheme']['test_dir']), 'r'):
    word = line.split(' ')[0].strip()
    if word!='':
        trwo.append(word)
        
RE_D = re.compile('\d')
def hasnum(string):
    res = RE_D.search(string) is not None
    if res:
        print (string)
    return 

nums = Counter(map(hasnum, trwo))
nums

5
51
107
90
83
173
143
1959
57
700
20
32
40
26
27
3.3
32
27
93
88
112
94
105
91
117
100
104
88
120
105
105
86
150
40
18
3
41.03
1
1
17
4
0
7,000
600
5400
1,008
600
1,000
300
0052
120
316
376
3
233
32
120
31.5
160
805
700
3.5
6.5
5
165
900
1,000
10
5
4
250
218
13
240
215
5
290
260
930
2,000
50
10
930
2,075
140
230
96
תח22שאחדים
16
650
300
450
2
60
232
2
50
150
ש22חת
113
0054
180
25
4,000
3,000
4
2
13
2
0
1-0
12
8
2
0
12
4-0
11
89
3
1
2
0
25
76
11
66
71
11
4
1
2
1
11
2
0
0
1
2
2
4
1
0
1
34
41
76
89
74
1
0
0
1
4
1
1
1
2
1
0
0
0
0
3
2
14
42
62
49
11
65
1
0
66
18
11
0-0
2
1
3
1
1
1
1
1
12
1
3
1
0
3
1
4
0
5
1
1
1
0
0
3
2
0
20
18
18
17
19
1
2
3
0
1
0
1
1
2
0
2
5
27
25
25
0
3
3
1
1
1
1
0
1
0
2
0
0
0
1
0
2
0
11
22
20
19
14
7
4
2
4
3
1
2
4
1
0
1
2
0
2
1
3
0
3
0
11
11
9
9
9
9
1982
27
36
90
139
18
19
24
28
12
6
22
1421
7
2840
13
4054
17
33
37
91
81
91
88
99
91
2-12
3
6-17
5
33
53
50
25
7
18
90
139
14
9
111
110
43
96
83
99
91
92
73
21
2
6
1
5
1
3
18
10
0
5
32
111
110
40
85
85
97
97
43
39
12
47
46
1

Counter({None: 16828})

In [82]:
1362/125669

0.010837995050489779

In [87]:
from collections import defaultdict

wds = defaultdict(list)
for n, ds in configs.datasets.items():
    for k in ds:
        if 'dev' in k or 'test' in k or 'train' in k:
            path = os.path.join(configs.data_folder, ds[k])
            print(path)
            for line in open(path, 'r'):
                word = line.split(' ')[0].strip()
                if word!='':
                    wds[n].append(word)
                
wds = {k: set(v) for k, v in wds.items()}


../NER/data/for_ncrf/morph_gold_train.bmes
../NER/data/for_ncrf/morph_gold_dev.bmes
../NER/data/for_ncrf/morph_gold_test.bmes
../NER/data/for_ncrf/token_gold_train_fix.bmes
../NER/data/for_ncrf/token_gold_dev_fix.bmes
../NER/data/for_ncrf/token_gold_test_fix.bmes
../NER/data/for_ncrf/token_gold_train_concat.bmes
../NER/data/for_ncrf/token_gold_dev_concat.bmes
../NER/data/for_ncrf/token_gold_test_concat.bmes
../NER/data/for_ncrf/morph_yap_dev.bmes
../NER/data/for_ncrf/morph_yap_test.bmes
../NER/data/for_ncrf/morph_yap_dev_pruned_all.bmes
../NER/data/for_ncrf/morph_yap_dev_pruned_non_o.bmes


In [88]:
wds.keys()

dict_keys(['gold_morpheme', 'gold_token_bioes', 'gold_token_raw', 'yap_morpheme', 'yap_morpheme_pruned_all', 'yap_morpheme_pruned_non_o'])

In [96]:
{k: len(v) for k,v in wds.items()}

{'gold_morpheme': 18187,
 'gold_token_bioes': 28713,
 'gold_token_raw': 28713,
 'yap_morpheme': 6480,
 'yap_morpheme_pruned_all': 3206,
 'yap_morpheme_pruned_non_o': 3206}

In [90]:
len(wds['gold_morpheme'] | wds['yap_morpheme'])

18465

In [91]:
len(wds['gold_morpheme'] | wds['yap_morpheme'] | wds['yap_morpheme_pruned_all'])

18465

In [95]:
len( wds['yap_morpheme_pruned_all'] | wds['yap_morpheme_pruned_non_o'])

3206